In [2]:
%pip install plotly
%pip install dash


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 59.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 69.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 50.7 MB/s eta 0:00:00:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install jupyter_dash

  Preparing metadata (setup.py) ... done
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=adb1f6992505217532cbc81ca89d50da2294bd1d4f3ab3c0f02b750f17d8f12b
  Stored in directory: /home/codespace/.cache/pip/wheels/21/0a/27/62f77273aa12fefb20b9b277f1e937f98ae16dbd04ffa2b93a
Successfully built retrying
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import dash_table
from dash import dcc , html, dash_table
from dash.dependencies import Input, Output

from dash.dash_table.Format import Format, Scheme, Sign, Symbol
from dash.dash_table import FormatTemplate
import pandas as pd
import sqlite3 as sl3
#print(plotly.version)

In [6]:
#JupyterDash.infer_jupyter_proxy_config()  #  Need this for binder to work.

In [2]:
conn = sl3.connect('fash.db')
qry = "select * from sales"
df = pd.read_sql(qry, con=conn)
#df.to_csv('fashion.csv',index=False)

prod_des = 100 *  \
            pd.crosstab(index = df['product'], columns=df['designer'],margins= False,normalize= 'index')

#columns =  [{"name": i, "id": i,'type':'numeric','format':Format(precision=4)} for i in prod_des.columns]
money = FormatTemplate.money(2)
columns =  [{"name": i, "id": i,'type':'numeric','format':money} for i in prod_des.columns]

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Fashion Demo"),
      dash_table.DataTable(
        id='table',
        columns=columns,
        data=prod_des.to_dict('records'),
        style_cell=dict(textAlign='right'),
        style_header=dict(backgroundColor="paleturquoise"),
        style_data=dict(backgroundColor="lavender")
    ),
    html.H2(),
     html.Label([
        "Choose brand",
        dcc.Dropdown(
            id='brand', clearable=False,
            value='Zegna', options=[{'label': c, 'value': c} for c in set(df['designer'])
            ]),
    html.H3(),     
    html.Label([
        "Choose price cutoff",
        dcc.Dropdown(
            id='prc', clearable=False,
            value=10.00, options=[{'label':10.00,'value':10.00},{'label':12.00,'value':12.00},{'label':20.00,'value':20.00}]),
    html.Div(children=[
        dcc.Graph(id="graph")
    ]),   
    ]),
])  ])  



In [3]:
@app.callback(Output('graph', 'figure'),Output('table','data'),Input("brand", "value"),Input("prc","value") )   
def stuff_figure(brand,prc):
    fash = df[(df['designer']== brand) & (df['price'] > float(prc))]
    fig = make_subplots(rows=1,cols=2,subplot_titles=('Fun and Games','Shits and Giggles'))
    fig.add_trace(go.Bar(x=fash['product'],y=fash['day']), row=1, col=1)
    fig.add_trace(go.Bar(x=fash['product'],y=fash['store'],orientation='h'), row=1, col=2)
    dfa = df[df['price'] > float(prc)] 
    prod_des = 100 * pd.crosstab(index = dfa['product'], columns=dfa['designer'],margins= False,normalize= 'index')
    return fig,prod_des.to_dict('records')
   
    


app.run_server(mode='inline')